In this kernel I tried to stack some of the high-performing single models and validate on a holdout set. I have been having trouble to  reach an agreement between local cv score and lb score. 

Some conclusions I believe its true:
1. Do not trust the cv score obtained through searching an optimal threshold, because this process of validating includes data leakage.
2. The random seed seems to have at least some significance.

Some remaining problems:
1. Validaing with a holdout set is not working in this kernel. I thought this was because the distribution of labels in test data and train data was significantly different, but that did not seem true either. So what is the exact problem?


some reference:

https://www.kaggle.com/shujian/single-rnn-with-4-folds-clr 

https://www.kaggle.com/bminixhofer/a-validation-framework-impact-of-the-random-seed/notebook

https://www.kaggle.com/gmhost/gru-capsule


Tell me if missed any

In [ ]:
import pandas as pd
import numpy as np
from tqdm import tqdm
tqdm.pandas()
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn import metrics
from sklearn.metrics import f1_score 

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import *
from keras.models import *
from keras import initializers, regularizers, constraints, optimizers, layers
from keras.initializers import *
from keras.optimizers import *
import keras.backend as K
from keras.callbacks import *
import tensorflow as tf
import os
import time
import gc
import re
from unidecode import unidecode

In [ ]:
train = pd.read_csv("../input/train.csv")
test = pd.read_csv("../input/test.csv")
print("Train shape : ", train.shape)
print("Test shape : ", test.shape)

**Text Preprocessing and Embedding**

In [ ]:
train["question_text"] = train["question_text"].str.lower()
test["question_text"] = test["question_text"].str.lower()

puncts = [',', '.', '"', ':', ')', '(', '-', '!', '?', '|', ';', "'", '$', '&', '/', '[', ']', '>', '%', '=', '#', '*', '+', '\\', '•',  '~', '@', '£', 
 '·', '_', '{', '}', '©', '^', '®', '`',  '<', '→', '°', '€', '™', '›',  '♥', '←', '×', '§', '″', '′', 'Â', '█', '½', 'à', '…', 
 '“', '★', '”', '–', '●', 'â', '►', '−', '¢', '²', '¬', '░', '¶', '↑', '±', '¿', '▾', '═', '¦', '║', '―', '¥', '▓', '—', '‹', '─', 
 '▒', '：', '¼', '⊕', '▼', '▪', '†', '■', '’', '▀', '¨', '▄', '♫', '☆', 'é', '¯', '♦', '¤', '▲', 'è', '¸', '¾', 'Ã', '⋅', '‘', '∞', 
 '∙', '）', '↓', '、', '│', '（', '»', '，', '♪', '╩', '╚', '³', '・', '╦', '╣', '╔', '╗', '▬', '❤', 'ï', 'Ø', '¹', '≤', '‡', '√', ]
def clean_text(x):

    x = str(x)
    for punct in puncts:
        x = x.replace(punct, f' {punct} ')
    return x


train["question_text"] = train["question_text"].apply(lambda x: clean_text(x))
test["question_text"] = test["question_text"].apply(lambda x: clean_text(x))

In [ ]:
## some config values 
embed_size = 300 # how big is each word vector
max_features = None # how many unique words to use (i.e num rows in embedding vector)
maxlen = 72 # max number of words in a question to use #99.99%

## fill up the missing values
X = train["question_text"].fillna("_na_").values
X_test = test["question_text"].fillna("_na_").values

## Tokenize the sentences
tokenizer = Tokenizer(num_words=max_features, filters='')
tokenizer.fit_on_texts(list(X))

X = tokenizer.texts_to_sequences(X)
X_test = tokenizer.texts_to_sequences(X_test)

## Pad the sentences 
X = pad_sequences(X, maxlen=maxlen)
X_test = pad_sequences(X_test, maxlen=maxlen)

## Get the target values
Y = train['target'].values

sub = test[['qid']]

In [ ]:
del train, test
gc.collect()

In [ ]:
word_index = tokenizer.word_index
max_features = len(word_index)+1
def load_glove(word_index):
    EMBEDDING_FILE = '../input/embeddings/glove.840B.300d/glove.840B.300d.txt'
    def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')
    embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(EMBEDDING_FILE) if o.split(" ")[0] in word_index)

    all_embs = np.stack(embeddings_index.values())
    emb_mean,emb_std = all_embs.mean(), all_embs.std()
    embed_size = all_embs.shape[1]

    embedding_matrix = np.random.normal(emb_mean, emb_std, (max_features, embed_size))
    for word, i in word_index.items():
        if i >= max_features: continue
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None: embedding_matrix[i] = embedding_vector
            
    return embedding_matrix 
    
def load_fasttext(word_index):    
    EMBEDDING_FILE = '../input/embeddings/wiki-news-300d-1M/wiki-news-300d-1M.vec'
    def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')
    embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(EMBEDDING_FILE) if len(o)>100 and o.split(" ")[0] in word_index )

    all_embs = np.stack(embeddings_index.values())
    emb_mean,emb_std = all_embs.mean(), all_embs.std()
    embed_size = all_embs.shape[1]

    embedding_matrix = np.random.normal(emb_mean, emb_std, (max_features, embed_size))
    for word, i in word_index.items():
        if i >= max_features: continue
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None: embedding_matrix[i] = embedding_vector

    return embedding_matrix

def load_para(word_index):
    EMBEDDING_FILE = '../input/embeddings/paragram_300_sl999/paragram_300_sl999.txt'
    def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')
    embeddings_index = dict(get_coefs(*o.split(" ")) for o in open(EMBEDDING_FILE, encoding="utf8", errors='ignore') if len(o)>100 and o.split(" ")[0] in word_index)

    all_embs = np.stack(embeddings_index.values())
    emb_mean,emb_std = all_embs.mean(), all_embs.std()
    embed_size = all_embs.shape[1]
    
    embedding_matrix = np.random.normal(emb_mean, emb_std, (max_features, embed_size))
    for word, i in word_index.items():
        if i >= max_features: continue
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None: embedding_matrix[i] = embedding_vector
    
    return embedding_matrix

In [ ]:
embedding_matrix_1 = load_glove(word_index)
#embedding_matrix_2 = load_fasttext(word_index)
embedding_matrix_3 = load_para(word_index)
embedding_matrix = np.mean((embedding_matrix_1, embedding_matrix_3), axis=0)  
#del embedding_matrix_1, embedding_matrix_3
#gc.collect()
np.shape(embedding_matrix)

In [ ]:
def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

def f1_smart(y_true, y_pred):
    args = np.argsort(y_pred)
    tp = y_true.sum()
    fs = (tp - np.cumsum(y_true[args[:-1]])) / np.arange(y_true.shape[0] + tp - 1, tp, -1)
    res_idx = np.argmax(fs)
    return 2 * fs[res_idx], (y_pred[args[res_idx]] + y_pred[args[res_idx + 1]]) / 2

# Defining the models

The 4 stacking models I used are:  GRU+Capsule with the mean of embeddings, RNN with attention layer, Conv1D with 400 features and window size 4, GRU+Capsule with only Glove

**GRU + Capsule Model**

In [ ]:
def squash(x, axis=-1):
    # s_squared_norm is really small
    # s_squared_norm = K.sum(K.square(x), axis, keepdims=True) + K.epsilon()
    # scale = K.sqrt(s_squared_norm)/ (0.5 + s_squared_norm)
    # return scale * x
    s_squared_norm = K.sum(K.square(x), axis, keepdims=True)
    scale = K.sqrt(s_squared_norm + K.epsilon())
    return x / scale

# A Capsule Implement with Pure Keras
class Capsule(Layer):
    def __init__(self, num_capsule, dim_capsule, routings=3, kernel_size=(9, 1), share_weights=True,
                 activation='default', **kwargs):
        super(Capsule, self).__init__(**kwargs)
        self.num_capsule = num_capsule
        self.dim_capsule = dim_capsule
        self.routings = routings
        self.kernel_size = kernel_size
        self.share_weights = share_weights
        if activation == 'default':
            self.activation = squash
        else:
            self.activation = Activation(activation)

    def build(self, input_shape):
        super(Capsule, self).build(input_shape)
        input_dim_capsule = input_shape[-1]
        if self.share_weights:
            self.W = self.add_weight(name='capsule_kernel',
                                     shape=(1, input_dim_capsule,
                                            self.num_capsule * self.dim_capsule),
                                     # shape=self.kernel_size,
                                     initializer='glorot_uniform',
                                     trainable=True)
        else:
            input_num_capsule = input_shape[-2]
            self.W = self.add_weight(name='capsule_kernel',
                                     shape=(input_num_capsule,
                                            input_dim_capsule,
                                            self.num_capsule * self.dim_capsule),
                                     initializer='glorot_uniform',
                                     trainable=True)

    def call(self, u_vecs):
        if self.share_weights:
            u_hat_vecs = K.conv1d(u_vecs, self.W)
        else:
            u_hat_vecs = K.local_conv1d(u_vecs, self.W, [1], [1])

        batch_size = K.shape(u_vecs)[0]
        input_num_capsule = K.shape(u_vecs)[1]
        u_hat_vecs = K.reshape(u_hat_vecs, (batch_size, input_num_capsule,
                                            self.num_capsule, self.dim_capsule))
        u_hat_vecs = K.permute_dimensions(u_hat_vecs, (0, 2, 1, 3))
        # final u_hat_vecs.shape = [None, num_capsule, input_num_capsule, dim_capsule]

        b = K.zeros_like(u_hat_vecs[:, :, :, 0])  # shape = [None, num_capsule, input_num_capsule]
        for i in range(self.routings):
            b = K.permute_dimensions(b, (0, 2, 1))  # shape = [None, input_num_capsule, num_capsule]
            c = K.softmax(b)
            c = K.permute_dimensions(c, (0, 2, 1))
            b = K.permute_dimensions(b, (0, 2, 1))
            outputs = self.activation(tf.keras.backend.batch_dot(c, u_hat_vecs, [2, 2]))
            if i < self.routings - 1:
                b = tf.keras.backend.batch_dot(outputs, u_hat_vecs, [2, 3])

        return outputs

    def compute_output_shape(self, input_shape):
        return (None, self.num_capsule, self.dim_capsule)

In [ ]:
def create_capsule(embedding_matrix):
    K.clear_session()       
    inp = Input(shape=(maxlen,))
    x = Embedding(max_features, embed_size, weights=[embedding_matrix], trainable=False)(inp)
    x = SpatialDropout1D(rate=0.2)(x)
    x = Bidirectional(CuDNNGRU(100, return_sequences=True, 
                                kernel_initializer=glorot_normal(seed=12300), recurrent_initializer=orthogonal(gain=1.0, seed=10000)))(x)

    x = Capsule(num_capsule=10, dim_capsule=10, routings=4, share_weights=True)(x)
    x = Flatten()(x)

    x = Dense(100, activation="relu", kernel_initializer=glorot_normal(seed=12300))(x)
    x = Dropout(0.5)(x)
    x = BatchNormalization()(x)

    x = Dense(1, activation="sigmoid")(x)
    model = Model(inputs=inp, outputs=x)
    model.compile(loss='binary_crossentropy', optimizer=Adam(),)
    return model

**RNN + attention layer + clr**

In [ ]:
class CyclicLR(Callback):
    def __init__(self, base_lr=0.001, max_lr=0.006, step_size=2000., mode='triangular',
                 gamma=1., scale_fn=None, scale_mode='cycle'):
        super(CyclicLR, self).__init__()

        self.base_lr = base_lr
        self.max_lr = max_lr
        self.step_size = step_size
        self.mode = mode
        self.gamma = gamma
        if scale_fn == None:
            if self.mode == 'triangular':
                self.scale_fn = lambda x: 1.
                self.scale_mode = 'cycle'
            elif self.mode == 'triangular2':
                self.scale_fn = lambda x: 1/(2.**(x-1))
                self.scale_mode = 'cycle'
            elif self.mode == 'exp_range':
                self.scale_fn = lambda x: gamma**(x)
                self.scale_mode = 'iterations'
        else:
            self.scale_fn = scale_fn
            self.scale_mode = scale_mode
        self.clr_iterations = 0.
        self.trn_iterations = 0.
        self.history = {}

        self._reset()

    def _reset(self, new_base_lr=None, new_max_lr=None,
               new_step_size=None):
        """Resets cycle iterations.
        Optional boundary/step size adjustment.
        """
        if new_base_lr != None:
            self.base_lr = new_base_lr
        if new_max_lr != None:
            self.max_lr = new_max_lr
        if new_step_size != None:
            self.step_size = new_step_size
        self.clr_iterations = 0.
        
    def clr(self):
        cycle = np.floor(1+self.clr_iterations/(2*self.step_size))
        x = np.abs(self.clr_iterations/self.step_size - 2*cycle + 1)
        if self.scale_mode == 'cycle':
            return self.base_lr + (self.max_lr-self.base_lr)*np.maximum(0, (1-x))*self.scale_fn(cycle)
        else:
            return self.base_lr + (self.max_lr-self.base_lr)*np.maximum(0, (1-x))*self.scale_fn(self.clr_iterations)
        
    def on_train_begin(self, logs={}):
        logs = logs or {}

        if self.clr_iterations == 0:
            K.set_value(self.model.optimizer.lr, self.base_lr)
        else:
            K.set_value(self.model.optimizer.lr, self.clr())        
            
    def on_batch_end(self, epoch, logs=None):
        
        logs = logs or {}
        self.trn_iterations += 1
        self.clr_iterations += 1

        self.history.setdefault('lr', []).append(K.get_value(self.model.optimizer.lr))
        self.history.setdefault('iterations', []).append(self.trn_iterations)

        for k, v in logs.items():
            self.history.setdefault(k, []).append(v)
        
        K.set_value(self.model.optimizer.lr, self.clr())
    

In [ ]:
class Attention(Layer):
    def __init__(self, step_dim,
                 W_regularizer=None, b_regularizer=None,
                 W_constraint=None, b_constraint=None,
                 bias=True, **kwargs):
        self.supports_masking = True
        self.init = initializers.get('glorot_uniform')

        self.W_regularizer = regularizers.get(W_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias
        self.step_dim = step_dim
        self.features_dim = 0
        super(Attention, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 3

        self.W = self.add_weight((input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        self.features_dim = input_shape[-1]

        if self.bias:
            self.b = self.add_weight((input_shape[1],),
                                     initializer='zero',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)
        else:
            self.b = None

        self.built = True

    def compute_mask(self, input, input_mask=None):
        return None

    def call(self, x, mask=None):
        features_dim = self.features_dim
        step_dim = self.step_dim

        eij = K.reshape(K.dot(K.reshape(x, (-1, features_dim)),
                        K.reshape(self.W, (features_dim, 1))), (-1, step_dim))

        if self.bias:
            eij += self.b

        eij = K.tanh(eij)

        a = K.exp(eij)

        if mask is not None:
            a *= K.cast(mask, K.floatx())

        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())

        a = K.expand_dims(a)
        weighted_input = x * a
        return K.sum(weighted_input, axis=1)

    def compute_output_shape(self, input_shape):
        return input_shape[0],  self.features_dim

In [ ]:
def create_lstm_atten(embedding_matrix):
    K.clear_session()
    inp = Input(shape=(maxlen,))
    x = Embedding(max_features, embed_size, weights=[embedding_matrix], trainable=False)(inp)
    x = SpatialDropout1D(0.1)(x)
    x = Bidirectional(CuDNNLSTM(40, return_sequences=True))(x)
    y = Bidirectional(CuDNNGRU(40, return_sequences=True))(x)
    
    atten_1 = Attention(maxlen)(x) # skip connect
    atten_2 = Attention(maxlen)(y)
    avg_pool = GlobalAveragePooling1D()(y)
    max_pool = GlobalMaxPooling1D()(y)
    
    conc = concatenate([atten_1, atten_2, avg_pool, max_pool])
    conc = Dense(16, activation="relu")(conc)
    conc = Dropout(0.1)(conc)
    outp = Dense(1, activation="sigmoid")(conc)    

    model = Model(inputs=inp, outputs=outp)
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=[f1])
    
    return model

**CNN**

In [ ]:
def create_cnn(embedding_matrix):
    K.clear_session()       
    inp = Input(shape=(maxlen,))
    x = Embedding(max_features, embed_size, weights=[embedding_matrix], trainable=False)(inp)
    x = SpatialDropout1D(rate=0.2)(x)
    
    conv1 = Conv1D(400, 4, activation = 'relu')(x)
    conv1 = GlobalMaxPool1D()(conv1)
    
    out = Dropout(0.5)(conv1)
    out = Dense(400, activation="relu", kernel_initializer=glorot_normal(seed=12300))(out)
    out = Dense(1, activation="sigmoid")(out)
    model = Model(inputs=inp, outputs=out)
    model.compile(loss='binary_crossentropy', optimizer=Adam(),)
    return model

# Splitting the data

X_val is never seen in the training process of the base models, so it should simulate the situation of prediciting unseen data.

In [ ]:
X_train, X_val, Y_train, Y_val = train_test_split(X, Y, random_state = 2018, test_size = 0.2)

In [ ]:
val_pred = {}
test_pred = {}

# Training

The training approach I took was to use the epoch with the least validation loss.
The optimal f1 score obtained in the process is probably overestimated, since we are overfitting the validation data to find an optimal threshold. Instead, I believe when tuning single models, threshold should be a hyperparamater worthy of considering as well. But since I am stacking these models later, I do not need to pick a threshold.

**Train with GRU + Capsule**

In [ ]:
filepath1="weights1_best.h5"
checkpoint = ModelCheckpoint(filepath1, monitor='val_loss', verbose=2, save_best_only=True, mode='min')
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.6, patience=1, min_lr=0.0001, verbose=2)
earlystopping = EarlyStopping(monitor='val_loss', min_delta=0.0001, patience=2, verbose=2, mode='auto')
callbacks = [checkpoint, reduce_lr]
model = create_capsule(embedding_matrix)
print(model.summary()) 
model.fit(X_train, Y_train, batch_size=512, epochs=6, validation_data=(X_val, Y_val), verbose=2, callbacks=callbacks, )
model.load_weights(filepath1)
y_pred = model.predict([X_val], batch_size=1024, verbose=2)
val_pred['model1'] = y_pred
y_test = model.predict([X_test], batch_size=1024, verbose=2)
test_pred['model1'] = y_test
f1_score, threshold = f1_smart(np.squeeze(Y_val), np.squeeze(y_pred))
print('Optimal F1: {:.4f} at threshold: {:.4f}'.format(f1_score, threshold))

**Train with attention LSTM**

In [ ]:
clr = CyclicLR(base_lr=0.001, max_lr=0.002,
               step_size=300., mode='exp_range',
               gamma=0.99994)
filepath2="weights2_best.h5"
checkpoint = ModelCheckpoint(filepath2, monitor='val_loss', verbose=2, save_best_only=True, mode='min')
callbacks = [checkpoint, clr]
model = create_lstm_atten(embedding_matrix)
print(model.summary()) 
model.fit(X_train, Y_train, batch_size=1024, epochs=8, validation_data=(X_val, Y_val), verbose=2, callbacks=callbacks,)
model.load_weights(filepath2)
y_pred = model.predict([X_val], batch_size=1024, verbose=2)
val_pred['model2'] = y_pred
y_test = model.predict([X_test], batch_size=1024, verbose=2)
test_pred['model2'] = y_test
f1_score, threshold = f1_smart(np.squeeze(Y_val), np.squeeze(y_pred))
print('Optimal F1: {:.4f} at threshold: {:.4f}'.format(f1_score, threshold))

**Train with Conv1D**

In [ ]:
filepath3="weights3_best.h5"
checkpoint = ModelCheckpoint(filepath3, monitor='val_loss', verbose=2, save_best_only=True, mode='min')
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.6, patience=1, min_lr=0.0001, verbose=2)
earlystopping = EarlyStopping(monitor='val_loss', min_delta=0.0001, patience=2, verbose=2, mode='auto')
callbacks = [checkpoint, reduce_lr]
model = create_cnn(embedding_matrix)
print(model.summary()) 
model.fit(X_train, Y_train, batch_size=512, epochs=10, validation_data=(X_val, Y_val), verbose=2, callbacks=callbacks, )
model.load_weights(filepath3)
y_pred = model.predict([X_val], batch_size=1024, verbose=2)
val_pred['model3'] = y_pred
y_test = model.predict([X_test], batch_size=1024, verbose=2)
test_pred['model3'] = y_test
f1_score, threshold = f1_smart(np.squeeze(Y_val), np.squeeze(y_pred))
print('Optimal F1: {:.4f} at threshold: {:.4f}'.format(f1_score, threshold))

**GRU + Capsul only Glove**

In [ ]:
filepath4="weights4_best.h5"
checkpoint = ModelCheckpoint(filepath4, monitor='val_loss', verbose=2, save_best_only=True, mode='min')
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.6, patience=1, min_lr=0.0001, verbose=2)
earlystopping = EarlyStopping(monitor='val_loss', min_delta=0.0001, patience=2, verbose=2, mode='auto')
callbacks = [checkpoint, reduce_lr]
model = create_capsule(embedding_matrix_1)
print(model.summary()) 
model.fit(X_train, Y_train, batch_size=512, epochs=6, validation_data=(X_val, Y_val), verbose=2, callbacks=callbacks, )
model.load_weights(filepath4)
y_pred = model.predict([X_val], batch_size=1024, verbose=2)
val_pred['model4'] = y_pred
y_test = model.predict([X_test], batch_size=1024, verbose=2)
test_pred['model4'] = y_test
f1_score, threshold = f1_smart(np.squeeze(Y_val), np.squeeze(y_pred))
print('Optimal F1: {:.4f} at threshold: {:.4f}'.format(f1_score, threshold))

# Ensembling

The model I used to stack the base models are lightgbm. I tried linear regression as well, which also boosted the score on holdout set but did not help a lot on the lb score.

In [ ]:
from sklearn.metrics import f1_score as f1

In [ ]:
len(val_pred)

In [ ]:
len(test_pred)

In [ ]:
stack_x = np.concatenate([i for i in val_pred.values()], axis = 1)
stack_test = np.concatenate([i for i in test_pred.values()], axis = 1)

In [ ]:
print('shape of training data: ', stack_x.shape)
print('shape of test data: ', stack_test.shape)

In [ ]:
y_test = np.zeros((56370,))

First, split the training data into train set and validation set. The training set is not seen by the base models, but seen by the stacking model. The validation set is not seen by any model, and therefore should be able to simulate the result of predicting on unseen data. Also the size of validation data is chosen to simulate the actual test data deciding lb score.

In [ ]:
stack_train, oof_train, stack_train_target, oof_target = train_test_split(stack_x, Y_val, random_state = 2018, test_size = 0.2)

In [ ]:
oof_train.shape

In [ ]:
oof_test = np.zeros(oof_train.shape[0])

In [ ]:
import lightgbm as lgb
params = {}
params['learning_rate'] = 0.05
params['boosting_type'] = 'gbdt'
params['objective'] = 'binary'
params['metric'] = 'binary_logloss'
params['sub_feature'] = 0.5
params['num_leaves'] = 10
params['min_data'] = 50
params['max_depth'] = 6

We train the model with 10-fold cross validation, and take the mean of each fold to predict the unseen data. As said above, it would be better to pick a universe threshold instead of finding the best threshold, in order to avoid data leakage. I chose the threshold as 0.36, becasue it turned out to give the best result.

In [ ]:
kfold = StratifiedKFold(n_splits=10, random_state=2018, shuffle=True)

In [ ]:
for i, (train_index, valid_index) in enumerate(kfold.split(stack_train, stack_train_target)):
    stack_x_train, stack_x_val, stack_y_train, stack_y_val = stack_train[train_index], stack_train[valid_index], stack_train_target[train_index], stack_train_target[valid_index]
    d_train = lgb.Dataset(stack_x_train, label=stack_y_train)
    clf = lgb.train(params, d_train, 100)
    stack_y_pred = (clf.predict(stack_x_val) > 0.36).astype(int)
    f1_score = f1(stack_y_val, stack_y_pred)
    print('f1 score for fold ', i, ': ', f1_score)
    y_test += (clf.predict(stack_test))/10
    oof_test += (clf.predict(oof_train))/10

The f1 score for a single fold might not be stable, but it would be much more robust if we combine the mean of results from 10 fold. The result on hold out set (which hasn't been seen by any model) would reach 0.705+ with threshold 0.36. There might be better thresholds, but once again we should not always pick the optimal threshold to avoid overfitting validation data.

In [ ]:
for i in [0.30, 0.31, 0.32, 0.33, 0.34, 0.35, 0.36, 0.37, 0.38]:
    print('threshold: ', i, "f1: ", f1(oof_target, (oof_test > i).astype(int)))

In [ ]:
y_test = y_test.reshape((-1, 1))
pred_test_y = (y_test>0.36).astype(int)
sub['prediction'] = pred_test_y
sub.to_csv("submission.csv", index=False)

# The Problem with Local CV Score

However, even the model performs well in predicting the unseen holdout set, its score on public lb is usually much lower (around 0.687) compared to the score obtained through local validation. What might cause the problem?

**Randomness**  
First, it might be possible that the model just happens to fit well on the holdout data I chose. Therefore I tried to simulate different holdout data with different random seeds. According to my results the random seeds do have some effect on the result (just like the threshold! but we should not use the best one, but rather a universal one to avoid data leakage)  But the result is consistently 0.70+, with minimum of 0.695.

In [ ]:
def tune_seed(seed):
    stack_train, oof_train, stack_train_target, oof_target = train_test_split(stack_x, Y_val, random_state = seed, test_size = 0.2)
    oof_test = np.zeros(oof_target.size)
    for i, (train_index, valid_index) in enumerate(kfold.split(stack_train, stack_train_target)):
        stack_x_train, stack_x_val, stack_y_train, stack_y_val = stack_train[train_index], stack_train[valid_index], stack_train_target[train_index], stack_train_target[valid_index]
        d_train = lgb.Dataset(stack_x_train, label=stack_y_train)
        clf = lgb.train(params, d_train, 100)
        oof_test += (clf.predict(oof_train))/10
    score = f1(oof_target, (oof_test > 0.34).astype(int))
    return score

In [ ]:
holdout_scores = []
for s in range(1999, 2020):
    holdout_score = tune_seed(s)
    print('f1 score at seed ', s, ' is ', holdout_score)

Seems that the difference is not due to randomness? I then thought that the distribution of labels in the lb test data is significantly different from the data provided. So I calculated the distribution.

In [ ]:
#proportion of of 1s in the entire labeled data
np.sum(Y) / len(Y)

In [ ]:
#number of 1s in the holdout set
np.sum(oof_target)/(len(oof_target))

In [ ]:
#proportion of 1s in predicted test set
np.sum(pred_test_y)/len(pred_test_y)

There seems to be a difference of about 0.007, but that might be due to my model mislabeling. So the distribution of lb test data is around the same as the labeld data provided.

The difference between lb score (0.687) and cv score (0.70+) should not be due to randomness or difference in label distribution.

**So, what's wrong with the local validation method? Or what's wrong with the lb test data?**